In [ ]:
#Mount Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


## Imports

In [ ]:
import os
import pandas as pd

In [ ]:
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.2 MB/s eta 0:00:00


## Load Dataset

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Owais/stock_price_after_call.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18375 entries, 0 to 18374
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    18375 non-null  object 
 1   exchange                18375 non-null  object 
 2   q                       18375 non-null  object 
 3   ticker                  18375 non-null  object 
 4   transcript              18375 non-null  object 
 5   date_only               18375 non-null  object 
 6   stock_price             16935 non-null  float64
 7   stock_price_after_call  14314 non-null  float64
dtypes: float64(2), object(6)
memory usage: 1.1+ MB


In [ ]:
df_full=df.dropna(subset=["stock_price","stock_price_after_call"])
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14220 entries, 0 to 18371
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    14220 non-null  object 
 1   exchange                14220 non-null  object 
 2   q                       14220 non-null  object 
 3   ticker                  14220 non-null  object 
 4   transcript              14220 non-null  object 
 5   date_only               14220 non-null  object 
 6   stock_price             14220 non-null  float64
 7   stock_price_after_call  14220 non-null  float64
dtypes: float64(2), object(6)
memory usage: 999.8+ KB


## Preprocessing

In [ ]:
!pip install contractions --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 2.7 MB/s eta 0:00:00


In [ ]:
#add column that includes the total character count for each earning call transcripts
df_full['char_count']= df_full['transcript'].str.len()
df_full[['transcript', 'char_count']].head()

<ipython-input-8-1b60e6d48d2a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full['char_count']= df_full['transcript'].str.len()


,transcript,char_count
0,"Prepared Remarks:\nOperator\nGood day, and wel...",35736
1,Prepared Remarks:\nOperator\nThank you for sta...,29761
2,Prepared Remarks:\nOperator\nGood day and welc...,53094
3,"Prepared Remarks:\nOperator\nGood day, everyon...",47532
4,Prepared Remarks:\nOperator\nGreetings and wel...,27273


In [ ]:
import contractions
import re
import nltk
import string
nltk.download('punkt')
from nltk import word_tokenize

def preprocess_text(text):
    if isinstance(text, str):

        text= re.sub(r'((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', ', ', text)

        text= ' '.join([word.strip() for word in text.split() if word.strip() and word[0] not in ['@', '#', '']])

        #remove quotes and apostrophes
        text=re.sub('[^\w\s]','',text)

        # clean up contractions
        text = contractions.fix(text)

        # lowercase text
        text = text.lower()

        # strip html from the text
        text = re.sub('<[^<]+?>', '', text)

        # strip out standalone digits
        text = re.sub(r'\b\d+\b', '', text)

        # tokenize text into individual terms
        tokens = word_tokenize(text)

        # pass each token through the function that determines if we keep it or not.
        clean_tokens = list(filter(None, tokens))
        return clean_tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df_full['preprocess_trans']= df_full['transcript'].apply(lambda x : ' '.join(preprocess_text(x)))

<ipython-input-10-204e70e876f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full['preprocess_trans']= df_full['transcript'].apply(lambda x : ' '.join(preprocess_text(x)))


In [ ]:
df_full['preprocess_trans'].sample(20)

13999    prepared remarks operator ladies and gentlemen...
519      prepared remarks operator ladies and gentlemen...
5055     prepared remarks operator good morning and wel...
5267     prepared remarks operator greetings welcome to...
5659     prepared remarks operator good afternoon my na...
9032     prepared remarks operator ladies and gentlemen...
4995     prepared remarks operator hello and welcome to...
6741     prepared remarks operator good afternoon my na...
2863     prepared remarks operator good day and thank y...
14648    prepared remarks operator ladies and gentlemen...
4903     prepared remarks operator welcome to the sunpo...
5695     prepared remarks operator good morning and wel...
9293     prepared remarks operator ladies and gentlemen...
16586    prepared remarks operator hello everyone and t...
3236     prepared remarks operator good morning my name...
2661     prepared remarks operator good afternoon and w...
1728     prepared remarks operator good day and welcome.

## Labeled

* Set 1 if the stock price increases else set 0 for decreasing

In [ ]:
df_full['change_indicator'] = (df_full['stock_price_after_call'] > df_full['stock_price']).astype(int)

<ipython-input-12-dfedba772e87>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full['change_indicator'] = (df_full['stock_price_after_call'] > df_full['stock_price']).astype(int)


In [ ]:
df_full.head(5)

,date,exchange,q,ticker,transcript,date_only,stock_price,stock_price_after_call,char_count,preprocess_trans,change_indicator
0,"Aug 27, 2020, 9:00 p.m. ET",NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel...",2020-08-27,45.139999,64.860001,35736,prepared remarks operator good day and welcome...,1
1,"Jul 30, 2020, 4:30 p.m. ET",NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...,2020-07-30,17.238564,17.968714,29761,prepared remarks operator thank you for standi...,1
2,"Oct 23, 2019, 5:00 p.m. ET",NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,2019-10-23,221.749268,298.385040,53094,prepared remarks operator good day and welcome...,1
3,"Nov 6, 2019, 12:00 p.m. ET",NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon...",2019-11-06,87.694138,80.516602,47532,prepared remarks operator good day everyone an...,0
4,"Aug 7, 2019, 8:30 a.m. ET",NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...,2019-08-07,14.448421,14.972457,27273,prepared remarks operator greetings and welcom...,1


In [ ]:
(df_full['stock_price_after_call'] == df_full['stock_price']).astype(int).value_counts()

0    14216
1        4
dtype: int64

In [ ]:
df_full['change_indicator'].value_counts()

0    7121
1    7099
Name: change_indicator, dtype: int64

## FINBERT

In [ ]:
df=df_full.copy()

In [ ]:
!pip install transformers --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.0 MB/s eta 0:00:00


In [ ]:
!pip install datasets --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
torch.cuda.is_available()

True

In [ ]:
df_=df[['preprocess_trans','change_indicator']].rename(columns = {'change_indicator':'label','preprocess_trans':'sentence'})

In [ ]:
df_train, df_test, = train_test_split(df_, stratify=df_['label'], test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train, stratify=df_train['label'],test_size=0.2, random_state=42)
print(df_train.shape, df_test.shape, df_val.shape)

(9100, 2) (2844, 2) (2276, 2)


In [ ]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=2)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain',do_lower_case=True)

In [ ]:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length' , max_length=128), batched=True)


Map:   0%|          | 0/9100 [00:00<?, ? examples/s]

Map:   0%|          | 0/2276 [00:00<?, ? examples/s]

Map:   0%|          | 0/2844 [00:00<?, ? examples/s]

In [ ]:
dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

args = TrainingArguments(
        output_dir = 'temp/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
)

trainer = Trainer(
        model=model,
        args=args,                  # training arguments, defined above
        train_dataset=dataset_train,         # training dataset
        eval_dataset=dataset_val,            # evaluation dataset
        compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.695000,0.681167,0.561072


Epoch,Training Loss,Validation Loss,Accuracy
1,0.695000,0.681167,0.561072
2,0.657000,0.678645,0.588313


TrainOutput(global_step=2276, training_loss=0.6788127946099414, metrics={'train_runtime': 542.4207, 'train_samples_per_second': 33.553, 'train_steps_per_second': 4.196, 'total_flos': 1197155301888000.0, 'train_loss': 0.6788127946099414, 'epoch': 2.0})

In [ ]:
model.eval()
trainer.predict(dataset_test).metrics

{'test_loss': 0.6831550002098083,
 'test_accuracy': 0.5759493670886076,
 'test_runtime': 21.9898,
 'test_samples_per_second': 129.333,
 'test_steps_per_second': 16.189}

### Another Trial ( Different model )

In [ ]:
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert',do_lower_case=True)

In [ ]:
import transformers
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf

# Load FinBERT model
finbert = TFAutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Get Bert embeddings and classifier layers
bert_embeddings = finbert.bert
classifier = finbert.classifier

# Recreate classifier with 2 units for binary classes
classifier = tf.keras.layers.Dense(2, activation='softmax')

# Define model
model = tf.keras.Sequential([
  bert_embeddings,
  classifier
])


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Compile model
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from transformers import BertConfig

# Load the pre-trained model configuration
config = BertConfig.from_pretrained('ProsusAI/finbert')
config


BertConfig {
  "_name_or_path": "/home/ubuntu/finbert/models/language_model/finbertTRC2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "negative",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 1,
    "neutral": 2,
    "positive": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.32.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    pipeline,
    Trainer,
    TrainingArguments
)
id2label= {0: "positive", 1: "negative"}
label2id = {"positive": 0, "negative": 1}

In [ ]:
# Load the pre-trained model
# model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')
model = AutoModelForSequenceClassification.from_pretrained(
  'ProsusAI/finbert', num_labels=2, id2label=id2label, label2id=label2id,ignore_mismatched_sizes=True
)
# Modify the classification head for binary classification
# model.classifier = torch.nn.Linear(model.config.hidden_size, 2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

args = TrainingArguments(
          output_dir = 'temp/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=4,
          eval_steps=500,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
)

trainer = Trainer(
        model=model,
        args=args,                  # training arguments, defined above
        train_dataset=dataset_train,         # training dataset
        eval_dataset=dataset_val,            # evaluation dataset
        compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.691975,0.518014
2,0.697900,0.683183,0.564148
3,0.697900,0.706863,0.545694
4,0.652000,0.726488,0.552724


TrainOutput(global_step=1140, training_loss=0.6680815111126817, metrics={'train_runtime': 899.342, 'train_samples_per_second': 40.474, 'train_steps_per_second': 1.268, 'total_flos': 2394310603776000.0, 'train_loss': 0.6680815111126817, 'epoch': 4.0})

In [ ]:
model.eval()
trainer.predict(dataset_test).metrics

{'test_loss': 0.6819029450416565,
 'test_accuracy': 0.5478199718706048,
 'test_runtime': 20.0442,
 'test_samples_per_second': 141.886,
 'test_steps_per_second': 4.44}